In [51]:
import os, random, shutil
from pathlib import Path

SPLIT_TREINO = 0.8   # 80 treino 20 teste
SEED = 42
EXTS = {".png", ".jpg", ".jpeg"}

BASE = Path.cwd()
IMAGENS = BASE / "imagens"
SRC = IMAGENS / "original"
DST_TRAIN = IMAGENS / "train"
DST_TEST  = IMAGENS / "test"
SPLIT = "train"  # "train" ou "test"
IMG_DIR   = IMAGENS / SPLIT
LABEL_DIR = IMAGENS / "labels" / SPLIT
LABEL_DIR.mkdir(parents=True, exist_ok=True)

random.seed(SEED)

subdirs = [p for p in SRC.iterdir() if p.is_dir()]
if subdirs:
    classes = sorted([d.name for d in subdirs])
else:
    classes = ["_"]  # classe única; imagens diretamente em 'original'

# Garante diretórios de saída
for c in classes:
    (DST_TRAIN / c).mkdir(parents=True, exist_ok=True)
    (DST_TEST  / c).mkdir(parents=True, exist_ok=True)

def listar_imagens(pasta: Path):
    return sorted([p for p in pasta.glob("*") if p.suffix.lower() in EXTS])

def copiar(lista, destino: Path):
    for src_path in lista:
        if classes == ["_"]:
            dst_path = destino / src_path.name
        else:
            dst_path = destino / src_path.parent.name / src_path.name
        if not dst_path.exists():
            shutil.copy2(src_path, dst_path)

for c in classes:
    pasta_classe = SRC if c == "_" else (SRC / c)
    imgs = listar_imagens(pasta_classe)
    idx = list(range(len(imgs)))
    random.shuffle(idx)
    corte = max(1, int(SPLIT_TREINO * len(idx))) if len(idx) > 1 else len(idx)
    train_idx, test_idx = idx[:corte], idx[corte:]
    train_imgs = [imgs[i] for i in train_idx]
    test_imgs  = [imgs[i] for i in test_idx]
    copiar(train_imgs, DST_TRAIN)
    copiar(test_imgs,  DST_TEST)
print("total:", len(imgs), "train:", len(train_imgs), "test:", len(test_imgs))


total: 14 train: 11 test: 3


In [52]:
# Classes (ordem importa; número da tecla associa a classe)
CLASSES = ["ruler", "waterline"]  # adicione mais se necessário

# Lista de imagens suportadas
EXTS = {".png", ".jpg", ".jpeg"}
images = sorted([p for p in IMG_DIR.glob("**/*") if p.suffix.lower() in EXTS])
if not images:
    raise SystemExit(f"Nenhuma imagem encontrada em {IMG_DIR}")

def yoloLabels(txt_path: Path, W, H):
    boxes = []
    if not txt_path.exists():
        return boxes
    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            c, cx, cy, w, h = map(float, parts)
            c = int(c)
            x1 = (cx - w/2.0) * W
            y1 = (cy - h/2.0) * H
            x2 = (cx + w/2.0) * W
            y2 = (cy + h/2.0) * H
            boxes.append([c, x1, y1, x2, y2])
    return boxes

def saveLabels(txt_path: Path, boxes, W, H):
    with open(txt_path, "w", encoding="utf-8") as f:
        for c, x1, y1, x2, y2 in boxes:
            x1, y1 = max(0, min(W-1, x1)), max(0, min(H-1, y1))
            x2, y2 = max(0, min(W-1, x2)), max(0, min(H-1, y2))
            w = (x2 - x1) / W
            h = (y2 - y1) / H
            cx = (x1 + x2) / (2*W)
            cy = (y1 + y2) / (2*H)
            f.write(f"{c} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

#editor
ix = iy = 0
drawing = False
current_box = None  # [x1,y1,x2,y2]
current_cls = 0
boxes = []          # lista de [cls,x1,y1,x2,y2]
idx = 0

print(f"total de imagens para rotular = {len(images)}")
print("primeira imagem:", images[0].name)


total de imagens para rotular = 11
primeira imagem: trescoroas_2025-11-05_15-10-11.jpg
